# Datové typy

V této části si popíšeme, co se v jazyce Python vlastně ukrývá za proměnnými, jak rozpoznává datové typy a jakým způsobem nakládá s pamětí.

![Python data structure](https://media.geeksforgeeks.org/wp-content/uploads/20191023173512/Python-data-structure.jpg)
*Obrázek z webu GeeksForGeeks: https://www.geeksforgeeks.org/python-data-types/*


In [12]:
a = 1024
print(a, type(a), id(a)) # tiskne value, type a id
b = 1025
print(b, type(b), id(b))

1024 <class 'int'> 139802668932400
1025 <class 'int'> 139802668941264


In [11]:
b = 765.12
print(b, type(b), id(b))

765.12 <class 'float'> 139802670772144


Všechny tyto vlastnosti můžeme i porovnávat. Operátor `==` porovnává hodnot, zatímco operátor `is`porovnává právě `id`. Rozlišujeme tím tak objekty s totožnou hodnotou a objekty identické.

In [14]:
a = 1000
b = 1000
print(a == b, a is b)

True False


In [15]:
a = 1000
b = a
print(a == b, a is b)

True True


## Implicitní a explicitní konverze

Mezi jednotlivými datovými typy může docházet ke konverzi. Pokud konverzi provede sám Python, protože je např. nutná k dokončení operace, hovoříme o **implicitní konverzi**. 

In [27]:
a = 2
b = 3.0
c = a + b
print(type(a), type(b), type(c))

<class 'int'> <class 'float'> <class 'float'>


In [29]:
c = 1 / 2
print(type(c))

<class 'float'>


Pokud konverzi vynutím e sami, pak se jedná o **explicitní koverzi**. Konverzi na daný typ explicitně vynucujeme voláním konstruktoru finálního typu, tj. funkcí se stejným názvem.

In [30]:
a = "1"
b = float(a)
c = int(a)
d = bool(a)

a, b, c, d

('1', 1.0, 1, True)

Je nutno dávat pozor, zda konverze skutečně vede k výsledku, který očekáváme - vždycky lepší vyzkoušet. Moje oblíbená chyba:

In [32]:
bool_flag = bool("t")
print(bool_flag)
bool_flag = bool("f")
print(bool_flag)

True
True


```{warning}
Interpret si nemusí vždycky myslet to samé co vy.
```

## Číselné typy a reprezentace čísel v počítači

Python nabízí tři základní číselné datové typy:

- `int` - integer reprezentující celá čísla,
- `float` - čísla s plovoucí desetinnou čárkou,
- `complex` - reprezentující komplexní čísla (fakticky dvakrát `float`)

Konceptuálně prosté, ale je dobré se podívat na pár vlastností souvisejících s reprezentací čísel v počítači. I v tomto má Python jistá specifika.

### Celá čísla

Celá čísla jsou v počítači reprezentována prostě - jedná se o přímočarý zápis ve dvojkové soustavě. Standardní int má běžně 32 bitů (4 byty), z nichž první je rezervován pro indikaci znaménka. Ukažme si, jak to funguje na typu char (to samé, jen s jedním bytem).

```c
char a = 127;
char b = a + 1;
printf("%d", b);
```

Kolik je 127+1? Správně, -128. Proč? Záporná čísla, tj. čísla s 1 v první bitu, se interpretují pomocí dodatečné operace `bitwise NOT`+1. Tedy prohodíme 0 a 1 a k výsledku přičteme číslo jedna. Ukažme si pár čísel v rozsahu jednoho byte a pochopíme. 

|base 10|  base 2 | NOT | two's complement |
|  ---  |   ---   | --- | --- |
| 1     | 00000001| |
| 10    | 00001010| |
| 16    | 00010000| |
| 127   | 01111111| |
| -127  | 10000001| 01111110 | 01111111 |
| -1    | 11111111| 00000000 | 00000001 |
| -2    | 11111110| 00000001 | 00000010 |
| -6    | 11111010| 00000101 | 00000110 |

Odsud už je vidět proč 127+1=-128: 
```
127=011111111
127+1=10000000
NOT(10000000)=01111111
NOT(10000000)+1=10000000
10000000=128
-> -128
```

```{note}
Jedná se o způsob reprezentace zvaný Two's complement. Detaily naleznete třeba na Wiki {cite:ts}`twos_complement`
```
Pro 32 bitový signed int obdobně zjistíte, že 2147483647+1=-2147483648. Chybám tohoto druhu se říká Overflow Error a mohou být velmi zrádné. Stály za řadou různě závažných nehod, uveďme si dvě

1. ESA Ariane flight V88 - přetekla hodnota `horizontal bias` ukládáná do 16-bit signed integer proměnné. Následná neošetřená hardware exception zablokovala část řízení a raketa se výrazně odchýlila od planované trasy. Let ukončil autodestrukční systém ({cite:ts}`esa_overflow`).
1. V roce 2012 vydala korejská hudební skupina Psy svůj velkolepý hit Gangnam Style. A již v roce 2013 překročil počet shlédnutí na YouTube `2 147 483 647`, čímž se dostal do záporných hodnot. Google tenkrát přešel od 32-bit signed integer pro reprezentaci počtu shlédnutí k 64-bit signed integer. Doposud se Psy počet `9 223 372 036 854 775 807` shlédnutí překonat nepodařilo. ({cite:ts}`psy_overflow`)

Proč o tom hovoříme? Protože Python je implementovaný v C, tak bychom mohli u typu int očekávam nějakou podobnou záludnost.

Ve skutečnosti Python používá C typ long, což je 8 bytová reprezentace celého čísla. Pokud připustíme unsigned variantu (tedy pouze kladná čísla, i první bit se použije), je maximální hodnota 2^64 - 1 = 18 446 744 073 709 551 615. Z toho, co jsme viděli víme, že navýšíme-li toto číslo o 1, dostaneme 0. Zkusme to v pythonu.

In [35]:
a = 18_446_744_073_709_551_615
b = a + 1
print(a)
print(b)

18446744073709551615
18446744073709551616


Dostali jsme správný výsledek. Už samotný int objekt je poměrně složitý a dokáže pracovat se složitější reprezentací celých čísel a v případě potřeby alokuje více paměti. Int v pythonu je neomezený.

### Floating point numbers

Následující (ne)rovnost naplňuje řadu lidí bázní, mnohým se hroutí svět a jiní v návalu paniky začínají obviňovat konkrétní programovací jazyky.

In [36]:
0.1 + 0.2 == 0.3

False

My budeme chytřejší a seznámíme se s tím, proč si **i Váš** počítač myslí, že

In [3]:
0.1+0.2

0.30000000000000004

Následující kód definuje několik pomocných funkcí, které zobrazují binární reprezentaci floating point čísel v počítači. Jejich implementace nás nemusí zajímat, věnujme se těm číslům. Z porovnání je zřejmé, že výsledky skutečně nejsou totožné.

In [1]:
import struct
def binary(num):
    return ''.join('{:0>8b}'.format(c) for c in struct.pack('!d', num))

def print_by_sections(num: str):
    print(num[0], num[1:12], num[12:16], "", end="")
    for i  in range(6):
        print(num[16+i*8:16+(i+1)*8], "", end="")
    print()
    
def show_repr(num):
    print_by_sections(binary(num))

show_repr(0.1)
show_repr(0.2)
show_repr(0.1+ 0.2)
show_repr(0.3)
show_repr(0.1+0.2-0.3)

0 01111111011 1001 10011001 10011001 10011001 10011001 10011001 10011010 
0 01111111100 1001 10011001 10011001 10011001 10011001 10011001 10011010 
0 01111111101 0011 00110011 00110011 00110011 00110011 00110011 00110100 
0 01111111101 0011 00110011 00110011 00110011 00110011 00110011 00110011 
0 01111001001 0000 00000000 00000000 00000000 00000000 00000000 00000000 


Reprezentace necelých čísel v počítači je výrazně složitější. Python používá variantu definovanou standardem IEE 754, která je vlastně binární obdobou toho, čemu říkáme scientific notation. Podle počtu použitých bitů rozlišujeme single (32 bitů) a double (64 bitů) precision floating point number.

![Wiki](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d2/Float_example.svg/590px-Float_example.svg.png)
![Wiki](https://upload.wikimedia.org/wikipedia/commons/thumb/a/a9/IEEE_754_Double_Floating_Point_Format.svg/618px-IEEE_754_Double_Floating_Point_Format.svg.png)

Výhodou této reprezentace je, že umožňuje uchovávat čísla v obrovském rozsahu hodnot: zhruba $10^{-38}$ až $10^{38}$ pro single precision, $10^{-308}$ až $10^{308}$ pro double precision. Python pro svůj typ `float` ve skutečnosti používá double precision.

Protože v double precision má mantisa (significand, tj. to, co není exponent) 52 bitů, dochází k ořezu. Obecně v single precision můžeme očekávat 7-8 platných číslic v desítkové soustave, v double precision 15-16. Některá čísla proste budou reprezentována s chybou. Tyto chyby se při operacích můžou kumulovat a je tak možné zdánlivě nevinným výpočtem dojít k úplně špatnému výsledku. Tomu, jak se takovým problémům vyhnout, se věnuje numerická matematika.

My si z toho vezmeme prozatím jednoduché ponaučení - nikdy nebudeme porovnávat `float` přímo, ale budeme se dívat, jestli jsou dostatečně blízko. Tedy něco jako

In [63]:
import math

a = 0.1 + 0.2
b = 0.3

if a == b:
    print("this will not work")

if abs(a-b) < 1e-15:
    print("this should work for small numbers")

if abs((a-b)/a) < 1e-15:
    print("this should work for large numbers")

if math.isclose(a, b, rel_tol=1e-15):
    print("this should be always safe")

this should work for small numbers
this should work for large numbers
this should be always safe


```{danger}
**POČÍTAČ NEUMÍ POČÍTAT!** Ale dělá to velmi rychle. Počítejme s tím.
```

## Bool

Na rozdíl od řady programovacích jazyků je v Pythonu samostatný logický typ bool. Může nabývat dvou různých hodnot:

```
True
False
```

Hodnoty typu bool jsou typicky výstupem z vyhodnocení logických výrazů, jako je např. porovnávání.

## Kolekce

Kontejnery (někdy kolekce) jsou takové datové typy, které mohou obsahovat další prvky. Možným kritériem pro jejich další dělení je uspořádání.

- Sekvence (řazené) – prvky mají definované pořadí, přistupujeme k nim pomocí indexů v hranaté závorce. Patří sem např. `list`, `tuple` nebo `string`.
- Neřazené – prvky nemají žádné konkrétní pořadí, indexování tady nemá smysl. Patří sem např. `set`.

Jiným příkladem kontejneru je `dict`, neboli dictionary, slovník – mapovací typ. Od verze Python 3.6 má definované pořadí, ale indexování jako sekvence nepodporuje.

### List

`list` (seznam) je druh mutable (měnitelné) sekvence, která se typicky používá jako obdoba dynamických polí.

Listy mohou obsahovat prvky různých typů. Zapisují se hranatou závorkou. Pomocí zabudovaných metod, jako `.append()`, nebo .`remove()` můžeme obsah listu měnit.

```
>>> my_list = [1, 2.71, "jogurt", True]
>>> my_list[0] = 2
>>> my_list.append(False)
>>> my_list.remove("jogurt")
>>> my_list
[2, 2.71, True, False]
```

### Tuple

`tuple` (n-tice) je immutable (neměnitelná) sekvence, která se typicky používá jako obdoba statických polí.

Tuples mohou obsahovat prvky různých typů. Zapisují se kulatou závorkou. Obsah tuple můžeme číst, ale nemůžeme měnit.

```
>>> my_tuple = (1, 2.71, "jogurt", True)
>>> my_list[2]
jogurt
>>> my_tuple[0] = 2  # -> TypeError
```

### Dictionary

Dictionary (slovník) je mutable (měnitelný) mapping, tedy typ, který obsahuje dvojice `key` a `value` (klíč a hodnota).

Slovníky mohou obsahovat prvky různých typů a mnoho typů lze i použít jako klíč. Zapisují se složenou závorkou, klíč a hodnotu odděluje dvojtečka. K hodnotám ve slovíku přistupujeme pomocí odpovídajících klíčů.


## String

String `str` (řetězec) je neměnitelná (immutable) sekvence znaků. Jakožto sekvence podporuje standardní indexování včetně sliců. Přestože je kolekcí, zaslouží se vlastní sekci.

Zapisuje se pomocí jednoduchých i dvojitých uvozovek.

```
>>> my_str = 'jogurt'
>>> my_str[3]
'u'
>>> my_str + 'y'
'jogurty'
>>> my_str[0] = "J"
>>> my_str
"Jogurty"
>>>
>>> my_str = "Příliš žluťoučký kůň úpěl ďábelské ódy"
>>> my_str[7:16]
'žluťoučký'
>>> my_str[::-1]
'ydó ékslebáď lěpú ňůk ýkčuoťulž šilířP'
```

Velikou výhodou typu string v Pythonu je, že v sobě má zabudované nepřeberné množství nejrůznějších operací.

Všechny najdete v dokumentaci. Namátkou jich pár vyberme:

```
>>> my_str = "Příliš žluťoučký kůň úpěl ďábelské ódy"
>>> my_str.split(" ")
['Příliš', 'žluťoučký', 'kůň', 'úpěl', 'ďábelské', 'ódy']
>>> "_".join(["hello", "world"]).upper()
'HELLO_WORLD'
>>> '###hello###'.strip('#')
'hello'
```

## NoneType

Významným datovým typem je `NoneType`, který slouží k reprezentaci "ničeho", tedy nedefinované, nepřiřazené nebo neexistující hodnoty, anglicky null value.

Jeho hodnotou je `None`, funkce `type` navrací `NoneType` a na typ `bool` se vždy konvertuje jako `False`. Bežně se s ním setkáme např. jako s _návratovou hodnotou funkcí, které nic nevrací_.

Zda je proměnná typu NoneType, porovnáváme pomocí operátoru `is`

In [10]:
def f():
    # this function does not return anything
    pass

a = f()

if a is None:
    print("there is nothing in var a")

there is nothing in var a


```{note}
Operátor `is` lze použít, neboť `NoneType` je implementován jako *singleton*, takže všechny instance `NoneType` jsou identické objekty. Vzpomeňte si na kurzy návrhových vzorů, které Vás možná teprve čekají.
```